<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src/CE_Transfer_Learning_AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [2]:
# dataset_root = '/content/drive/MyDrive/Research/Crowd Emotion/Spectrograms/Log/'

In [3]:
train_dir = '/content/drive/MyDrive/Research/Crowd Emotion v2/Spectrograms/Erb/Train/'
test_dir = '/content/drive/MyDrive/Research/Crowd Emotion v2/Spectrograms/Erb/Test/'

In [4]:
# dataset v1
# dataset = datasets.ImageFolder(
#     dataset_root,
#     transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         # transforms.Normalize([0, 0, 0], [255, 255, 255])
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])
# )

# dataset v1
# dataset_size = len(dataset)
# test_size = int(0.2 * dataset_size)
# train_size = dataset_size - test_size

# train_dataset, test_dataset = torch.utils.data.random_split(
#         dataset, [train_size, test_size]
# )



In [5]:
train_dataset = datasets.ImageFolder(
    train_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # transforms.Normalize([0, 0, 0], [255, 255, 255])
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

test_dataset = datasets.ImageFolder(
    test_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # transforms.Normalize([0, 0, 0], [255, 255, 255])
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [6]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

In [7]:
model = models.alexnet(pretrained=True)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)
model.cuda()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
NUM_EPOCHS = 20
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('[ %02d ]: Accuracy: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

[ 00 ]: Accuracy: 0.634500
[ 01 ]: Accuracy: 0.596072
[ 02 ]: Accuracy: 0.617421
[ 03 ]: Accuracy: 0.597780
[ 04 ]: Accuracy: 0.598634
[ 05 ]: Accuracy: 0.596072
[ 06 ]: Accuracy: 0.652434
[ 07 ]: Accuracy: 0.700256
[ 08 ]: Accuracy: 0.627669
[ 09 ]: Accuracy: 0.672929
[ 10 ]: Accuracy: 0.678907
[ 11 ]: Accuracy: 0.677199
[ 12 ]: Accuracy: 0.678907
[ 13 ]: Accuracy: 0.690863
[ 14 ]: Accuracy: 0.710504
[ 15 ]: Accuracy: 0.675491
[ 16 ]: Accuracy: 0.710504
[ 17 ]: Accuracy: 0.660120
[ 18 ]: Accuracy: 0.682323
[ 19 ]: Accuracy: 0.731853


In [10]:
"""
                Train           Test
        Mel      --             0.697
        Bark     --             0.948
        Log      --             0.912
        Erb      --             0.731

"""

'\n                Train           Test\n        Mel      --             0.697\n        Bark     --             0.948\n        Log      --             0.912\n        Erb      --             \n\n'